In [1]:
from utils import EyeTrackingDataLoader
from transformers import AutoTokenizer
import pandas as pd
import os

### Script parameters
- Language: supported 'it' and 'en'
- Model name: the model to use
- Rollout: if to apply rollout to ValueZeroing
- Aggregation method: how to compute the attention of a word composed of multiple sub-tokens. Supported values:
	- sum: sum attention values of all sub-tokens
	- max: take the maximum attention value among sub-tokens
	- mean: average the attention values of the sub-tokens
	- first: take the attention value of the first sub-token
- Layer: the layer from which to extract attention

In [2]:
language = 'en'
rollout = True
aggregation_method = 'sum'
layer = 11
model_name = 'xlm-roberta-base'

In [3]:
eye_tracking_data_dir = f'../augmenting_nlms_meco_data/{language}'

out_dir = f'data/value_zeroing/{language}/{model_name}'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

out_file_name = f'{aggregation_method}' if not rollout else f'{aggregation_method}_rollout'
out_file_name += f'_l{layer}.json'
out_path = os.path.join(out_dir, out_file_name)
out_path

'data/value_zeroing/en/xlm-roberta-base/sum_rollout_l11.json'

In [4]:
dl = EyeTrackingDataLoader(eye_tracking_data_dir)
sentences_df = dl.load_sentences()

In [5]:
sentences_df

,sent_id,sentence
0,1_1,"[In, ancient, Roman, religion, and, myth,, Jan..."
1,1_2,"[He, has, a, double, nature, and, is, usually,..."
2,1_3,"[Janus, presided, over, the, beginning, and, e..."
3,1_4,"[The, doors, of, his, temple, were, open, in, ..."
4,1_5,"[As, the, god, of, gates,, he, was, also, asso..."
...,...,...
96,12_4,"[The, registration, identifier, is, a, series,..."
97,12_5,"[In, some, countries,, the, identifier, is, un..."
98,12_6,"[France, was, the, first, country, to, introdu..."
99,12_7,"[Early, twentieth, century, plates, varied, in..."


In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
subword_prefix = '▁'  # non ho trovato un modo di prenderlo automaticamente dal tokenizer

Create a mapping of indices to align model's tokens and sub-tokens to original words

In [53]:
def align_to_original_words(model_tokens:list , original_tokens:list, subword_prefix:str) -> list:
    model_tokens = model_tokens[1: -1]                  # Remove <s> and </s>
    aligned_model_tokens = []
    alignment_ids = []
    alignment_id = -1
    orig_idx = 0
    for token in model_tokens:
        alignment_id += 1
        if token.startswith(subword_prefix):            # Remove the sub-word prefix
            token = token[len(subword_prefix):]         
        if len(aligned_model_tokens) == 0:              # First token (serve?)
            aligned_model_tokens.append(token)
        elif aligned_model_tokens[-1] + token in original_tokens[orig_idx]: # We are in the second (third, fourth, ...) sub-token
            aligned_model_tokens[-1] += token                               # so we merge the token with its preceding(s)
            alignment_id -= 1
        else:
            aligned_model_tokens.append(token)
        if aligned_model_tokens[-1] == original_tokens[orig_idx]:   # A token was equal to an entire original word or a set of 
            orig_idx += 1                                           # sub-tokens was merged and matched an original word
        alignment_ids.append(alignment_id)

    if aligned_model_tokens != original_tokens:
        raise Exception(f'Failed to align tokens.\nOriginal tokens: {original_tokens}\nObtained alignment: {aligned_model_tokens}')
    return alignment_ids

In [60]:
def create_subwords_alignment(sentences_df: pd.DataFrame, tokenizer: AutoTokenizer, subword_prefix: str) -> dict:
    sentence_alignment_dict = dict()

    for idx, row in sentences_df.iterrows():
        sent_id = row['sent_id']
        sentence = row['sentence']
        tokenized_sentence = tokenizer(sentence, is_split_into_words=True, return_tensors='pt')
        input_ids = tokenized_sentence['input_ids'].tolist()[0] # 0 because the batch_size is 1
        model_tokens = tokenizer.convert_ids_to_tokens(input_ids) 
        aligned_tokens, alignment_ids = align_to_original_words(model_tokens, sentence, subword_prefix)
        sentence_alignment_dict[sent_id] = {'model_input': tokenized_sentence, 'alignment_ids': alignment_ids}
    return sentence_alignment_dict

In [61]:
sentence_alignment_dict = create_subwords_alignment(sentences_df, tokenizer, subword_prefix)

In [ ]:
from transformers import AutoModelForMaskedLM

class TokenContributionExtractor():
    
    def __init__(self, model_name:str, layer:int, rollout:bool, aggregation_method:str):
        self.layer = layer
        self.rollout = rollout
        self.aggregration_method = aggregation_method
        self.model = self._load_model(model_name)
    
    def _load_model(self, model_name:str):
        """
        Left abstract since some contribution extraction methods have to modify the default model architecture
        """
    
    